In [15]:
import taichi as ti
ti.init(
    random_seed=42,
    arch=ti.cpu,
    debug=1,
    advanced_optimization=0,
    excepthook=False,
    cpu_max_num_threads=1
)

import IPython
import numpy as np

# Matplotlib settings.
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim

# %matplotlib notebook
%matplotlib inline

[Taichi] Starting on arch=x64


## Optimization Through Dynamics

In [16]:
# ti helper funcs
dim = 2
real = ti.f32

scalar = lambda **kwargs: ti.field(dtype=real, shape=1, **kwargs)
vector = lambda **kwargs: ti.Vector.field(dim, dtype=real, **kwargs)

In [17]:
# init ti vars
# ti.reset() # reset ti kernel to be allowed to init variables

dt = scalar(needs_grad=False)
steps = 1

height = 225
width_to_height_ratio = 16/9.
width = int(width_to_height_ratio * height)
DY = 1/height # difference is inverse of smaller of height, width

num_species = 4
particles_per_species = 20
N = num_species * particles_per_species
Ndim = N * dim

lr = 0.01 # learning rate

params_need_grad = True

# particle parameters
repelling_force = scalar(needs_grad=params_need_grad) # force active if r < separation radius
temperature = scalar(needs_grad=params_need_grad) # controls random fluctuations in particle velocities -> increases gradient variance
friction_coef = scalar(needs_grad=params_need_grad)  
separation_radius = scalar(needs_grad=params_need_grad) # mean separation radius
interaction_radius = scalar(needs_grad=params_need_grad) # mean interaction radius
force_strength = scalar(needs_grad=params_need_grad) # inter-particle force strength
close_range_factor = scalar(needs_grad=params_need_grad) # force strength multiplier at r=0
dist_range_factor = scalar(needs_grad=params_need_grad) # force strength multiplier at r=self.height
stdev = scalar(needs_grad=params_need_grad) # spread in species parameters -> increases gradient variance
seed_range = scalar(needs_grad=params_need_grad) # initial position spread

params = {
    "dt": dt,
    "repelling_force": repelling_force,
    "temperature": temperature,
    "friction_coef": friction_coef,
    "separation_radius": separation_radius,
    "interaction_radius": interaction_radius,
    "force_strength": force_strength,
    "close_range_factor": close_range_factor,
    "dist_range_factor": dist_range_factor,
    "stdev": stdev,
}
P = len(params)

array_needs_grad = True
species     = np.array([[spec]*particles_per_species for spec in range(num_species)]).reshape(-1)
force_radii = scalar(needs_grad=array_needs_grad)
separation_ = scalar(needs_grad=array_needs_grad)
intrprtcl_f = scalar(needs_grad=array_needs_grad)
repulsive_f = scalar(needs_grad=array_needs_grad)

# FIXME: I abuse ti.fields as N x 2 position/speed/force arrays. problem?
# also abuse them as adjacency matrix (Dxy)

# Game state:
R = vector(needs_grad=array_needs_grad)
V = vector(needs_grad=array_needs_grad)
F = vector(needs_grad=array_needs_grad)

# loss
complexity = scalar(needs_grad=True)

arrays = {
    "force_radii": force_radii,
    "separation_": separation_,
    "intrprtcl_f": intrprtcl_f,
    "repulsive_f": repulsive_f,
    "R": R,
    "V": V,
    "F": F,
    "complexity": complexity
}

In [18]:
# place ti vars

# ti.root.deactivate_all()
#print(ti.root.get_children())
#print(dir(ti.root))

# place scalars
ti.root.place(*(list(params.values()) + [complexity]))

# neural network example:
# ti.root.dense(ti.ij, (n_actuators, n_sin_waves)).place(weights)
# ti.root.dense(ti.i, n_actuators).place(bias)

# place N x 2 vectors
# e.g. V = [steps, N] x 2
ti.root.dense(ti.l, steps).dense(ti.i, N).place(
    V, R, F
)
ti.root.dense(ti.ij, (N, N)).place(
    force_radii, separation_, intrprtcl_f, repulsive_f
)

ti.root.lazy_grad()

RuntimeError: [snode_expr_utils.cpp:place_child@43] This variable has been placed.

In [19]:
# initialization kernels
idx = lambda spec, prtcl: num_species * prtcl + spec

@ti.kernel
def set_ti_scalars():
    idx0d = 0
    dt[idx0d] = 0.01
    seed_range[idx0d] = 0.9 # initial position spread

    # set particle parameters
    repelling_force[idx0d] = 4.0 # force active if r < separation_radius
    temperature[idx0d] = 20.0 # controls random fluctuations in particle velocities -> increases gradient variance
    friction_coef[idx0d] = 90.0  
    separation_radius[idx0d] = 25.0 # mean separation radius
    interaction_radius[idx0d] = 25.0 # mean interaction radius
    force_strength[idx0d] = 10.0 # inter-particle force strength
    close_range_factor[idx0d] = 2.0 # force strength multiplier at r=0
    dist_range_factor[idx0d] = 2.0 # force strength multiplier at r=self.height
    stdev[idx0d] = 0.05 # spread in species parameters -> increases gradient variance
    
    # scalar objective function
    complexity[idx0d] = 0.0

@ti.func
def increment_vector_inplace(array_vector: ti.template(), magnitude: float, dy: float, dx: float):
    # increment array_vector (which is/may be row in a [? x 2]) vector field)
    theta = ti.atan2(dy, dx)
    ti.atomic_add(array_vector[0], magnitude * ti.cos(theta))
    ti.atomic_add(array_vector[1], magnitude * ti.sin(theta))

@ti.kernel
def set_ti_vectors():
    # takes in R, V, F and updates them, t should be 0
    t: ti.i32 = 0
    for prtcl in range(N):
        center_x = width/2
        center_y = height/2
        
        R[t, prtcl][0] = ti.random() * width # + width/(2*seed_range)
        R[t, prtcl][1] = ti.random() * height # + height/(2*seed_range)
        V[t, prtcl][0] = ti.random() * 2 - 1 # -1 to 1
        V[t, prtcl][1] = ti.random() * 2 - 1
    
@ti.func
def particle_assignment_loop(spec_a, spec_b, fr, sep, f, rep):
    for prtcl_a in range(particles_per_species):
        for prtcl_b in range(particles_per_species):
            # assign
            force_radii[idx(spec_a, prtcl_a), idx(spec_b, prtcl_b)] = fr
            separation_[idx(spec_a, prtcl_a), idx(spec_b, prtcl_b)] = sep
            intrprtcl_f[idx(spec_a, prtcl_a), idx(spec_b, prtcl_b)] = f
            repulsive_f[idx(spec_a, prtcl_a), idx(spec_b, prtcl_b)] = rep
    
@ti.kernel
def set_block_matrices():
    # parameter matrices
    for spec_a in range(num_species):
        for spec_b in range(num_species):
            # sample once for this species pair direction
            
            # (reparameterization)
            fr = ti.abs((ti.randn() * stdev + 1) * interaction_radius)
            sep = ti.abs((ti.randn() * stdev + 1) * separation_radius)
            f = ti.randn() + force_strength
            rep_force = ti.abs((ti.randn() * stdev + 1) * repelling_force)
            
            particle_assignment_loop(spec_a, spec_b, fr_sep, f, rep)

In [20]:
# helper funcs

@ti.func
def wrap_borders(x: ti.template(), y: ti.template()):
    x_ = x % width
    y_ = y % height
    return x_, y_

@ti.kernel
def compute_complexity(t: ti.i32):
    # just macro temperature for now; FIXME make grid out of this
    # params R, V, F
    for prtcl in range(N):
        vx = V[t, prtcl][0]
        vy = V[t, prtcl][1]
        vsquare = vx * vx + vy * vy
        
        ti.atomic_add(complexity, vsquare / Ndim)

@ti.kernel
def apply_grad():
    # gradient ascent on scalar parameters
    # params[i][None] -= lr * params[i].grad[None] # for loop doesnt seem to work
    repelling_force[None] += lr * repelling_force.grad[None]
    temperature[None] += lr * temperature.grad[None]
    friction_coef[None] += lr * friction_coef.grad[None]
    separation_radius[None] += lr * separation_radius.grad[None]
    interaction_radius[None] += lr * interaction_radius.grad[None]
    force_strength[None] += lr * force_strength.grad[None]
    close_range_factor[None] += lr * close_range_factor.grad[None]
    dist_range_factor[None] += lr * dist_range_factor.grad[None]
    stdev[None] += lr * stdev.grad[None]

In [21]:
# physical kernels: attract, friction, update_system

@ti.kernel
def attract(t: ti.i32):
    # params F, R (read from t-1)
    # updates F (write to t)
    for a in range(N): # <-- gets vectorized <3
        for b in range(N): # <--- gets serialized => slowdown :-(

            ra, rb = R[t-1, a], R[t-1, b]
            
            dx, dy = ra[0] - rb[0], ra[1] - rb[1]
    
            dx, dy = wrap_borders(dx, dy)
            
            eps = 1e-5
            xdoty = dx * dx + dy * dy
            dr = ti.sqrt(xdoty) # euclidean distance
            inv_dr_2 = 1/(xdoty+eps) 

            f = close_range_factor * (inv_dr_2)
            # increase force at long range again
            f += (dist_range_factor - close_range_factor) * (dr * DY)
            f *= intrprtcl_f[a, b] # weight by randomly sampled interparticle force (depends on species)
            
            # add contributions from each particle b to entry a
            increment_vector_inplace(F[t, a], f, dy, dx)
            
@ti.kernel
def friction(t: ti.i32):
    # params V, R (reads from t-1)
    # updates F (writes to t)
    for prtcl in range(N):
        vx, vy = V[t-1, prtcl]
        v = ti.sqrt(vx * vx + vy * vy)
        theta = ti.atan2(vy, vx)

        # random doesnt seem to work in tape scope
        ffx = friction_coef[None] * v * ti.cos(theta) # + ti.randn() * temperature 
        ffy = friction_coef[None] * v * ti.sin(theta) # + ti.randn() * temperature

        ti.atomic_add(F[t, prtcl][0], ffx)
        ti.atomic_add(F[t, prtcl][0], ffx)

        
@ti.kernel
def update_system(t: ti.i32):
    # params R, V (reads from t-1), F (reads from t); dt 
    # updates R, V (writes to t)
    
    # this loop gets vectorized:
    for prtcl in range(N):
        x = R[t-1, prtcl][0]
        y = R[t-1, prtcl][1]
        print("update time=",t,"x=",x, "y=",y)
        vx = V[t-1, prtcl][0]
        vy = V[t-1, prtcl][1]
        dx = vx * dt
        dy = vy * dt

        xnew, ynew = x + dx, y + dy
        xnew, ynew = wrap_borders(xnew, ynew)
        R[t, prtcl][0] = xnew
        R[t, prtcl][1] = ynew
        
        fx = F[t, prtcl][0]
        fy = F[t, prtcl][1]
        
        V[t, prtcl][0] = vx - fx * dt
        V[t, prtcl][1] = vy - fy * dt

In [22]:
# main animate/forward functions that call into ti.kernels
GUI = 0
if GUI:
    gui = ti.GUI('Diff', res=(width, height))

def step(t: ti.i32):
    # simulation
    
    # calculate particle forces
    attract(t)
    # calculate friction
    # friction(t)
    # update positions and forces
    update_system(t)
           
    if GUI:
        # TODO add color channels,  see colorfolor examples
        grid = np.zeros((width, height))
        for prtcl in range(N):
            x = int(R[t, prtcl][0])
            y = int(R[t, prtcl][1])
            
            grid[x, y] += 20
        gui.set_image(grid)
        gui.show()
    else:
        particles_display = np.zeros(shape=(N, dim), dtype=np.float32)
        for prtcl in range(N):
            particles_display[prtcl, 0] = R[t, prtcl][0]
            particles_display[prtcl, 1] = R[t, prtcl][1]
        
        plt.xlim([0,width])
        plt.ylim([0,height])
        plt.scatter(particles_display[:,0], particles_display[:,1], marker=".", c="b")
        plt.show() # show each frame, comment to show only final frame
    
def run():
    # do simulation for many steps
    print(f"Simulating {steps} steps ...")
    if GUI:
        t = 1
        while gui.running and not gui.get_event(gui.ESCAPE) and t <= steps:
            step(t)
            t += 1
    else:
        for t in range(1,steps+1):
            step(t)
    
def report():
    for name, p in params.items():
        print(f"Param {name} grad: {p.grad.to_numpy().nonzero()}")
    for name, arr in arrays.items():
        print(f"{name}\t grad: {arr.grad.to_numpy().nonzero()}")
        print(f"{name}\t     : {arr.to_numpy().nonzero()}")
    print(f"Complexity: {complexity.to_numpy().nonzero()}")

# this function calls into the ti.kernels:
def update_params():
    set_ti_scalars()
    set_ti_vectors()
    report()
    # within this context: update param.grad using autodiff of complexity w.r.t each param
    with ti.Tape(complexity):
        run() # simulate
        compute_complexity(steps-1) 
        # ^ setting complexity score
        # v and then exiting ti.Tape scope sets param.grad
    apply_grad() # gradient ascent update
    # report()


In [23]:
update_params()

RuntimeError: [snode_tree.cpp:check_tree_validity@15] dense node must have at least one child.

In [24]:
"""
Taichi code requirements:
1. no continue/break in loops
2. only either one for loop or a group of other statements per indentation level
3. no recursive assignments of form x = f(x)
6. some index updating requirements, see documentation, ti.atomic_add
4. init all ti.Vector/ti.Field vars outside of kernel
5. all taichi code must be called into from a kernel function (decorated with @ti.kernel),
    which may call into inner taichi functions (decorated with @ti.func)

# for grad stuff, see
# https://docs.taichi.graphics/docs/lang/articles/advanced/differentiable_programming
6. the set_* kernels above must be outside of the tape scope, presumably inside tape scope the params
    w.r.t. which I autodiff may not be written to anymore by me? It could also be that its just
    the ti.random calls (those dont work in friction either!)
"""

'\nTaichi code requirements:\n1. no continue/break in loops\n2. only either one for loop or a group of other statements per indentation level\n3. no recursive assignments of form x = f(x)\n6. some index updating requirements, see documentation, ti.atomic_add\n4. init all ti.Vector/ti.Field vars outside of kernel\n5. all taichi code must be called into from a kernel function (decorated with @ti.kernel),\n    which may call into inner taichi functions (decorated with @ti.func)\n\n# for grad stuff, see\n# https://docs.taichi.graphics/docs/lang/articles/advanced/differentiable_programming\n6. the set_* kernels above must be outside of the tape scope, presumably inside tape scope the params\n    w.r.t. which I autodiff may not be written to anymore by me? It could also be that its just\n    the ti.random calls (those dont work in friction either!)\n'

In [25]:
# anim.FuncAnimation?

In [26]:
ti.GUI?

Init signature:
ti.GUI(
    name='Taichi',
    res=512,
    background_color=0,
    show_gui=True,
    fullscreen=False,
    fast_gui=False,
)
Docstring:     
Taichi Graphical User Interface class.

Args:
    name (str, optional): The name of the GUI to be constructed.
        Default is 'Taichi'.
    res (Union[int, List[int]], optional): The resolution of created
        GUI. Default is 512*512.
    background_color (int, optional): The background color of creted GUI.
        Default is 0x000000.
    show_gui (bool, optional): Specify whether to render the GUI. Default is True.
    fullscreen (bool, optional): Specify whether to render the GUI in
        fullscreen mode. Default is False.
    fast_gui (bool, optional): Specify whether to use fast gui mode of
        Taichi. Default is False.

Returns:
    :class:`~taichi.misc.gui.GUI` :The created taichi GUI object.
File:           ~/fun/simulation/simz/lib/python3.8/site-packages/taichi/misc/gui.py
Type:           type
Subclasses:  